In [9]:
%%capture
from pathlib import Path
from ccdproc import ImageFileCollection
from photozpy import Telescope, HeaderCorrection, Telescope, SwiftDownload, mImageFileCollection
import pandas as pd
import copy
from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
import shutil

## Download data

In [11]:
# set to the directory as you desire. It's the root directory of this notebook in default
dir = Path("./data")
swift_download = SwiftDownload(download_dir = dir)

# parse the obsquery infor from a csv file
swift_download.set_obsquery_info_from_csv("source_catalog.csv")

# download data
swift_download.download_swift_data(organize = True)

00038456003 on 2024-02-15 23:24:02 is being downloaded, the uvot mode is 0x30ed


00038456003 has been downloaded/examined, skipping ...
00014051002 on 2021-02-10 11:02:02 is being downloaded, the uvot mode is 0x30ed


00014051001 on 2021-02-07 16:01:02 is being downloaded, the uvot mode is 0x30ed


00014051001 has been downloaded/examined, skipping ...
00014051001 has been downloaded/examined, skipping ...
00016150001 on 2023-07-26 23:41:02 is being downloaded, the uvot mode is 0x30ed


00016150001 has been downloaded/examined, skipping ...


## Sum extensions within and across observations

In [23]:
# Get the directory for each source
paths = [path for path in dir.iterdir() if path.is_dir()]
print(paths)

# Hidden directories will be included if exists, make sure you remove the directory that you don't want.
paths.remove(paths[2])
print(paths)

[PosixPath('data/B3 0850+443'), PosixPath('data/87GB 213913.0+293303'), PosixPath('data/.ipynb_checkpoints'), PosixPath('data/4FGL J0700.5-6610')]
[PosixPath('data/B3 0850+443'), PosixPath('data/87GB 213913.0+293303'), PosixPath('data/4FGL J0700.5-6610')]


In [25]:
# Define a multi image collection object
multi_collection = mImageFileCollection(image_dir = paths)

In [47]:
len(multi_collection[0].files_filtered(include_path = True, filter = "UVM3"))

0

In [44]:
multi_collection[0].location.parts[-1]

'B3 0850+443'